In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [3]:
 from dotenv import load_dotenv
 load_dotenv()

True

In [4]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")

books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [7]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   lineterminator= "\n",
                                   index = False,
                                   header = False)

In [8]:
loader = TextLoader("tagged_description.txt", encoding= "utf-8")
raw_documents= loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
documents = text_splitter.split_documents(raw_documents)
print(f"Loaded {len(documents)} chunks.")
print(f"Total characters: {len(raw_documents[0].page_content)}")


Loaded 3655 chunks.
Total characters: 2612889


In [9]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [10]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

response = client.embeddings.create(
    input="This is a test",
    model="text-embedding-ada-002"
)
print(response)


CreateEmbeddingResponse(data=[Embedding(embedding=[-0.008058104664087296, -0.0036456480156630278, -0.0003476669080555439, -0.0056320796720683575, -0.024551108479499817, 0.016169093549251556, -0.014860229566693306, -0.00461076945066452, -0.0009791688062250614, -0.03434775397181511, 0.015944339334964752, 0.017253201454877853, -0.009142214432358742, 0.0020740199834108353, 0.00875219888985157, 5.1540635467972606e-05, 0.024233808740973473, 0.00035365758230909705, 0.008448119275271893, -0.007575544063001871, 0.005427156575024128, 0.007410283200442791, -0.011654174886643887, 0.024603992700576782, -0.02821328304708004, -0.023295128718018532, 0.0035597123205661774, -0.03535253927111626, 0.019421420991420746, -0.009895802475512028, 0.021933380514383316, -0.017292864620685577, 0.0015972432447597384, -0.03641020506620407, 0.000726320140529424, -0.012652348726987839, -0.010457688011229038, -0.017279643565416336, 0.008130819536745548, -0.01098652184009552, 0.009056278504431248, 0.016631823033094406,

In [11]:


embed_model = OpenAIEmbeddings()

test_embedding = embed_model.embed_query("Test embedding please")

print("Embedding vector length:", len(test_embedding))


Embedding vector length: 1536


In [12]:
from langchain.embeddings import OpenAIEmbeddings
from tqdm import tqdm

# Define the small subset
small_docs = documents[:200]  # Use first 200 chunks

# Initialize embedder
embedder = OpenAIEmbeddings()

# Test embedding loop
vectors = []
for doc in tqdm(small_docs, desc="Embedding"):
    vectors.append(embedder.embed_query(doc.page_content))



C:\Users\Alles Gut\AppData\Local\Temp\ipykernel_32076\3802884481.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedder = OpenAIEmbeddings()
Embedding: 100%|██████████| 200/200 [01:11<00:00,  2.81it/s]


In [14]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()
texts = [doc.page_content for doc in documents]

from tqdm import tqdm
vectors = [embedder.embed_query(text) for text in tqdm(texts, desc="Embedding All Chunks")]



Embedding All Chunks: 100%|██████████| 3655/3655 [21:06<00:00,  2.89it/s]
